In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import time

import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer
import qiskit as qiskit
import qiskit.quantum_info as qi
from qiskit import QuantumCircuit, transpile, Aer, IBMQ

import ramsey_experiment

# Loading your IBM Quantum account(s)
#IBMQ.save_account(os.environ.get('IBM_KEY'), overwrite=True)
#IBMQ.load_account()
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Options
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='ibm-q/open/main',
    token='4cb4dd43af6af16cc340156cf1dfebb32ec9f233cec56a0c6e98e9a574f99d96c7a4867d4541dc9394c58f81efe019ee4feb8fa17b475f79952b0f66bf000f4c'
)

# Or save your credentials on disk.
# QiskitRuntimeService.save_account(channel='ibm_quantum', instance='ibm-q/open/main', token='<IBM Quantum API key>')


C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [5]:
n = 1
circuits = []
backend = service.least_busy(operational=True, simulator=False)

delay = np.arange(0, 300, 1)
for d in delay:
    circuit = QuantumCircuit(n)
    for i in range(0, n, 2):
        circuit.h(i)
        circuit.delay(int(d),i, 'us')
    circuit.barrier()
    for i in range(0, n, 2):
        circuit.h(i)
    #circuit.measure_all()
    transpiled_circ = transpile(circuit, backend, scheduling_method='alap')
    circuits.append(transpiled_circ)
#circuits[100].draw('mpl')

In [3]:
from qiskit.quantum_info import Pauli

ZIII = Pauli('ZIII')
IZII = Pauli('IZII')
IIZI = Pauli('IIZI')
IIIZ = Pauli('IIIZ')
Z = 'Z'+'I'*(126)
Z = Pauli(Z)

In [6]:
from qiskit.tools import job_monitor

service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.least_busy(operational=True, simulator=False)
#backend = service.get_backend("ibmq_qasm_simulator")
options = Options()
options.resilience_level = 0
options.optimization_level = 0
#use only qubit number 8
initial_layout = [8]
estimator = Estimator(backend, options=options)
job = estimator.run(circuits=circuits, observables=[Z]*300, shots = 400, initial_layout=initial_layout)
#job = backend.run(circuits,400)
job_monitor(job)

#job = execute(circuits, backend, shots=4321)
#job.result()


KeyboardInterrupt



In [ ]:
plt.xlim([0, 3000])
plt.plot(delay, job.result().values)

In [ ]:
from scipy.signal import find_peaks

T = 3000
measurements = 300
a = job.result().values
b = a[::-1]
extended = np.concatenate((a,b))
fft_output_ext = np.fft.fft(extended)

import numpy as np
import matplotlib.pyplot as plt

# Set the sampling rate based on measurements
sample_rate = measurements / T

# Compute the positive frequencies for the FFT output
frequencies_ext = np.fft.fftfreq(2 * len(job.result().values), 1 / sample_rate)
frequencies_ext*=(2 * np.pi)
positive_indices = np.where(frequencies_ext > 0)
positive_magnitudes = np.abs(fft_output_ext)[positive_indices]


frequencies  = frequencies_ext[positive_indices]
weights =  np.abs(fft_output_ext)[positive_indices]

# Calculating cumulative weights
cumulative_weights = np.cumsum(weights)

# Finding the median position
total_weight = np.sum(weights)
half_total_weight = total_weight / 2

median_index = np.where(cumulative_weights >= half_total_weight)[0][0]
weighted_median_frequency = frequencies[median_index]

print(f'Weighted median frequency: {weighted_median_frequency}')
# Plot and annotate results
plt.plot(frequencies_ext, np.abs(fft_output_ext))
#plt.axvline(x=W[k], color='orange', linestyle='--',label=f'W{k} = {W[k]}')
plt.axvline(x=weighted_median_frequency, color='green', linestyle='--',label=f'W_found{weighted_median_frequency} ')

#a = L[k]

## find peaks ##
peaks, _ = find_peaks(positive_magnitudes)
peak_magnitudes = positive_magnitudes[peaks]
sorted_peak_indices = np.argsort(peak_magnitudes)[::-1]
n_highest_peaks = sorted_peak_indices[:3]
highest_freq = frequencies_ext[positive_indices][peaks[n_highest_peaks[0]]]
##


plt.legend()
plt.show()